# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 1 2023 10:32AM,258409,15,30014403,"Alliance Pharma, Inc.",Released
1,Mar 1 2023 10:32AM,258409,15,30014402,"Alliance Pharma, Inc.",Released
2,Mar 1 2023 10:31AM,258445,15,9220899,Lonza,Released
3,Mar 1 2023 10:27AM,258441,19,9220889,"Quagen Pharmaceuticals, LLC",Released
4,Mar 1 2023 10:26AM,258440,15,VT80184619,"Virtus Pharmaceuticals, LLC",Released
5,Mar 1 2023 10:20AM,258438,19,GRST30123,"Graystone, LLC",Released
6,Mar 1 2023 10:19AM,258437,10,CTF0011663,"Citieffe, Inc.",Released
7,Mar 1 2023 9:45AM,258435,10,0086464700,ISDIN Corporation,Released
8,Mar 1 2023 9:45AM,258435,10,0086464701,ISDIN Corporation,Released
9,Mar 1 2023 9:45AM,258435,10,0086465007,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,258437,Released,1
25,258438,Released,1
26,258440,Released,1
27,258441,Released,1
28,258445,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258437,NaN,NaN,1.0
258438,NaN,NaN,1.0
258440,NaN,NaN,1.0
258441,NaN,NaN,1.0
258445,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258268,0.0,18.0,32.0
258356,0.0,0.0,1.0
258357,0.0,2.0,27.0
258368,2.0,6.0,5.0
258375,0.0,0.0,56.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258268,0,18,32
258356,0,0,1
258357,0,2,27
258368,2,6,5
258375,0,0,56


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258268,0,18,32
1,258356,0,0,1
2,258357,0,2,27
3,258368,2,6,5
4,258375,0,0,56


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258268,,18,32
1,258356,,,1
2,258357,,2,27
3,258368,2,6,5
4,258375,,,56


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc."
2,Mar 1 2023 10:31AM,258445,15,Lonza
3,Mar 1 2023 10:27AM,258441,19,"Quagen Pharmaceuticals, LLC"
4,Mar 1 2023 10:26AM,258440,15,"Virtus Pharmaceuticals, LLC"
5,Mar 1 2023 10:20AM,258438,19,"Graystone, LLC"
6,Mar 1 2023 10:19AM,258437,10,"Citieffe, Inc."
7,Mar 1 2023 9:45AM,258435,10,ISDIN Corporation
72,Mar 1 2023 9:45AM,258434,10,ISDIN Corporation
106,Mar 1 2023 9:34AM,258375,15,"Alliance Pharma, Inc."
162,Mar 1 2023 9:33AM,258268,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc.",,,2
1,Mar 1 2023 10:31AM,258445,15,Lonza,,,1
2,Mar 1 2023 10:27AM,258441,19,"Quagen Pharmaceuticals, LLC",,,1
3,Mar 1 2023 10:26AM,258440,15,"Virtus Pharmaceuticals, LLC",,,1
4,Mar 1 2023 10:20AM,258438,19,"Graystone, LLC",,,1
5,Mar 1 2023 10:19AM,258437,10,"Citieffe, Inc.",,,1
6,Mar 1 2023 9:45AM,258435,10,ISDIN Corporation,,,65
7,Mar 1 2023 9:45AM,258434,10,ISDIN Corporation,,,34
8,Mar 1 2023 9:34AM,258375,15,"Alliance Pharma, Inc.",,,56
9,Mar 1 2023 9:33AM,258268,15,"Person & Covey, Inc.",,18,32


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc.",2,,
1,Mar 1 2023 10:31AM,258445,15,Lonza,1,,
2,Mar 1 2023 10:27AM,258441,19,"Quagen Pharmaceuticals, LLC",1,,
3,Mar 1 2023 10:26AM,258440,15,"Virtus Pharmaceuticals, LLC",1,,
4,Mar 1 2023 10:20AM,258438,19,"Graystone, LLC",1,,
5,Mar 1 2023 10:19AM,258437,10,"Citieffe, Inc.",1,,
6,Mar 1 2023 9:45AM,258435,10,ISDIN Corporation,65,,
7,Mar 1 2023 9:45AM,258434,10,ISDIN Corporation,34,,
8,Mar 1 2023 9:34AM,258375,15,"Alliance Pharma, Inc.",56,,
9,Mar 1 2023 9:33AM,258268,15,"Person & Covey, Inc.",32,18,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc.",2,,
1,Mar 1 2023 10:31AM,258445,15,Lonza,1,,
2,Mar 1 2023 10:27AM,258441,19,"Quagen Pharmaceuticals, LLC",1,,
3,Mar 1 2023 10:26AM,258440,15,"Virtus Pharmaceuticals, LLC",1,,
4,Mar 1 2023 10:20AM,258438,19,"Graystone, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc.",2.0,NaN,NaN
1,Mar 1 2023 10:31AM,258445,15,Lonza,1.0,NaN,NaN
2,Mar 1 2023 10:27AM,258441,19,"Quagen Pharmaceuticals, LLC",1.0,NaN,NaN
3,Mar 1 2023 10:26AM,258440,15,"Virtus Pharmaceuticals, LLC",1.0,NaN,NaN
4,Mar 1 2023 10:20AM,258438,19,"Graystone, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 1 2023 10:32AM,258409,15,"Alliance Pharma, Inc.",2.0,0.0,0.0
1,Mar 1 2023 10:31AM,258445,15,Lonza,1.0,0.0,0.0
2,Mar 1 2023 10:27AM,258441,19,"Quagen Pharmaceuticals, LLC",1.0,0.0,0.0
3,Mar 1 2023 10:26AM,258440,15,"Virtus Pharmaceuticals, LLC",1.0,0.0,0.0
4,Mar 1 2023 10:20AM,258438,19,"Graystone, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2842462,151.0,12.0,4.0
12,258421,1.0,0.0,0.0
15,1291937,92.0,18.0,0.0
16,258425,1.0,0.0,0.0
19,1033713,4.0,23.0,2.0
22,258417,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2842462,151.0,12.0,4.0
1,12,258421,1.0,0.0,0.0
2,15,1291937,92.0,18.0,0.0
3,16,258425,1.0,0.0,0.0
4,19,1033713,4.0,23.0,2.0
5,22,258417,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,151.0,12.0,4.0
1,12,1.0,0.0,0.0
2,15,92.0,18.0,0.0
3,16,1.0,0.0,0.0
4,19,4.0,23.0,2.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,151.0
1,12,Released,1.0
2,15,Released,92.0
3,16,Released,1.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,22
Status,,,,,,
Completed,4.0,0.0,0.0,0.0,2.0,0.0
Executing,12.0,0.0,18.0,0.0,23.0,0.0
Released,151.0,1.0,92.0,1.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,22
0,Completed,4.0,0.0,0.0,0.0,2.0,0.0
1,Executing,12.0,0.0,18.0,0.0,23.0,0.0
2,Released,151.0,1.0,92.0,1.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,22
0,Completed,4.0,0.0,0.0,0.0,2.0,0.0
1,Executing,12.0,0.0,18.0,0.0,23.0,0.0
2,Released,151.0,1.0,92.0,1.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()